# Analyse AirBnB Data from Seattle - Udacity Data Sience Nanodegree

**Goal:**
Analyse data according the CRISP-DM (Cross Industry Standard Process for Data Mining) method.

**Data source:**

Seattle AirBnb Data is available in kaggle: https://www.kaggle.com/airbnb/seattle
- calendar.csv
- listings.csv
- reviews.csv

# 1. Business understanding

Interesting Questions on dataset:
 
a) What neighbourhoods are most booked in Seattle

b) What kind of seasonal effects exist on pricing? Correlation to occupancy ration in the whole city?

c) What is the impact of higher reviews on the number of bookings? What kind of impact factor is more important, price or reviews?

d)  What are the most occuring amenities? What effect do they have on number of bookings?

# 2. Data Understanding


In [61]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
# read calendar.csv
df_cal = pd.read_csv('./data/calendar.csv')
print('Rows: ', df_cal.shape[0], 'Columns: ', df_cal.shape[1])
df_cal.head()

Rows:  1393570 Columns:  4


,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [3]:
df_cal.dtypes

listing_id     int64
date          object
available     object
price         object
dtype: object

In [4]:
# read listings.csv
df_list = pd.read_csv('./data/listings.csv')
print('Rows: ', df_list.shape[0], 'Columns: ', df_list.shape[1])
df_list.head()


Rows:  3818 Columns:  92


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [45]:
# count number of listing_ids
len(df_list.id.unique())

0         241032
1         953595
2        3308979
3        7421966
4         278830
          ...   
3813     8101950
3814     8902327
3815    10267360
3816     9604740
3817    10208623
Name: id, Length: 3818, dtype: int64


In [5]:
# column names + types
list_type = df_list.dtypes
pd.set_option('display.max_rows', 100)
print(list_type)


id                                    int64
listing_url                          object
scrape_id                             int64
last_scraped                         object
name                                 object
summary                              object
space                                object
description                          object
experiences_offered                  object
neighborhood_overview                object
notes                                object
transit                              object
thumbnail_url                        object
medium_url                           object
picture_url                          object
xl_picture_url                       object
host_id                               int64
host_url                             object
host_name                            object
host_since                           object
host_location                        object
host_about                           object
host_response_time              

In [6]:
# read reviews.csv
df_rev = pd.read_csv('./data/reviews.csv')
print('Rows: ', df_rev.shape[0], 'Columns: ', df_rev.shape[1])
df_rev.head()

Rows:  84849 Columns:  6


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


In [7]:
df_rev.dtypes

listing_id        int64
id                int64
date             object
reviewer_id       int64
reviewer_name    object
comments         object
dtype: object

**Result of data understanding:**



| Table | No of rows | No of columns | Content|
| :-: | :-:| :-: | :-: |
| *calendar* | 1393570 | 4 | availability and price for each object depending on date |
| *listing* | 3818 | 92 |  information for each object (host, neighborhood, amenities, type), 1 entry for each object |
| *reviews* | 84849 | 9 | comments of reviewers for each object |

## 3. Data Preparation

### dataframe calendar:
- convert datatypes
- remove NaN in column price
- extract year, month day 

In [8]:
# clean data in dataframe calendar
pd.set_option('display.max_row', 1000)
df_cal
#df_cal.dtypes

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN
...,...,...,...,...
1393565,10208623,2016-12-29,f,NaN
1393566,10208623,2016-12-30,f,NaN
1393567,10208623,2016-12-31,f,NaN
1393568,10208623,2017-01-01,f,NaN


In [52]:
# convert data type of column available to boolean
df_cal['available'] = df_cal['available'] == 't'

# convert data type of column price to float
df_cal['price'] = df_cal['price'].str.replace('$', '')


df_cal['price'] = df_cal['price'].str.replace(',', '')
df_cal['price'] = df_cal['price'].astype(float)

# convert date to datatype datatime & extract year, day, month, week
df_cal['year'] = pd.DatetimeIndex(df_cal['date']).year
df_cal['day'] = pd.DatetimeIndex(df_cal['date']).day
df_cal['month'] = pd.DatetimeIndex(df_cal['date']).month
df_cal['week'] = pd.DatetimeIndex(df_cal['date']).week
df_cal['date'] = pd.to_datetime(df_cal['date'])


AttributeError: Can only use .str accessor with string values!

In [10]:
# check if price is only NaN in case available is false
print(df_cal['available'].value_counts())
print(np.sum(df_cal.price.isna()))

# replace all na with 0
df_cal['price'].fillna(0, inplace=True)

True     934542
False    459028
Name: available, dtype: int64
459028


In [11]:
# final types
df_cal.dtypes

listing_id             int64
date          datetime64[ns]
available               bool
price                float64
year                   int64
day                    int64
month                  int64
dtype: object

### dataframe listing:

- copy all relevant columns to a new dataframe
- clean dataset
- check if values are 0 or NaN
- split categorical values to binär values if necessary

In [39]:
# column neighbourhood (relevant for 1a)

print("rows with null: ", np.sum(df_list['neighbourhood_group_cleansed'].isnull()), " --- rows with NaN:", np.sum(df_list['neighbourhood_group_cleansed'].isna()))
df_list['neighbourhood_group_cleansed'].value_counts()

#change to categorical variable
df_list['neighbourhood_group_cleansed'] = df_list['neighbourhood_group_cleansed'].astype('category')
print(df_list['neighbourhood_group_cleansed'])

rows with null:  0  --- rows with NaN: 0
0                Queen Anne
1                Queen Anne
2                Queen Anne
3                Queen Anne
4                Queen Anne
               ...         
3813    Other neighborhoods
3814           Capitol Hill
3815         Rainier Valley
3816           Capitol Hill
3817             Queen Anne
Name: neighbourhood_group_cleansed, Length: 3818, dtype: category
Categories (17, object): ['Ballard', 'Beacon Hill', 'Capitol Hill', 'Cascade', ..., 'Rainier Valley', 'Seward Park', 'University District', 'West Seattle']


**Column amenities**

For easier data analysis for each amenity a separate column with boolean value should be created

In [63]:
# column amenities (relevant for question 1d)
df_ame = df_list['amenities'].copy()
print(df_ame)


0       {TV,"Cable TV",Internet,"Wireless Internet","A...
1       {TV,Internet,"Wireless Internet",Kitchen,"Free...
2       {TV,"Cable TV",Internet,"Wireless Internet","A...
3       {Internet,"Wireless Internet",Kitchen,"Indoor ...
4       {TV,"Cable TV",Internet,"Wireless Internet",Ki...
                              ...                        
3813    {TV,"Cable TV",Internet,"Wireless Internet","A...
3814    {TV,"Cable TV",Internet,"Wireless Internet",Ki...
3815    {"Cable TV","Wireless Internet",Kitchen,"Free ...
3816    {TV,"Wireless Internet",Kitchen,"Free Parking ...
3817    {TV,"Cable TV",Internet,"Wireless Internet",Ki...
Name: amenities, Length: 3818, dtype: object


In [78]:
df_ame = df_ame.str.replace('"', '')
df_ame = df_ame.str.replace('{', '')
df_ame = df_ame.str.replace('}', '')
df_ame.fillna('none')
# print(df_ame)

# use mulitlabelbinarizer from sklearn: 
# https://stackoverflow.com/questions/58871608/how-to-binary-encode-multi-valued-categorical-variable-in-pandas
mlb = MultiLabelBinarizer()
df_label_ame = pd.DataFrame(mlb.fit_transform(df_ame.str.split(",")), columns = mlb.classes_, index=df_ame.index)
print(df_label_ame.columns)
print(df_label_ame)

Index(['', '24-Hour Check-in', 'Air Conditioning', 'Breakfast',
       'Buzzer/Wireless Intercom', 'Cable TV', 'Carbon Monoxide Detector',
       'Cat(s)', 'Dog(s)', 'Doorman', 'Dryer', 'Elevator in Building',
       'Essentials', 'Family/Kid Friendly', 'Fire Extinguisher',
       'First Aid Kit', 'Free Parking on Premises', 'Gym', 'Hair Dryer',
       'Hangers', 'Heating', 'Hot Tub', 'Indoor Fireplace', 'Internet', 'Iron',
       'Kitchen', 'Laptop Friendly Workspace', 'Lock on Bedroom Door',
       'Other pet(s)', 'Pets Allowed', 'Pets live on this property', 'Pool',
       'Safety Card', 'Shampoo', 'Smoke Detector', 'Smoking Allowed',
       'Suitable for Events', 'TV', 'Washer', 'Washer / Dryer',
       'Wheelchair Accessible', 'Wireless Internet'],
      dtype='object')
         24-Hour Check-in  Air Conditioning  Breakfast  \
0     0                 0                 1          0   
1     0                 0                 0          0   
2     0                 0               

In [38]:
df_listing = df_list[['id','neighbourhood_group_cleansed']]
#rename some columns
#df_listing.rename(columns={'neighbourhood_group_cleansed': 'neighbourhood'})
df_listing = df_listing.rename(index=str, columns={"neighbourhood_group_cleansed": "neighbourhood"})
df_listing


,id,neighbourhood
0,241032,Queen Anne
1,953595,Queen Anne
2,3308979,Queen Anne
3,7421966,Queen Anne
4,278830,Queen Anne
...,...,...
3813,8101950,Other neighborhoods
3814,8902327,Capitol Hill
3815,10267360,Rainier Valley
3816,9604740,Capitol Hill


## 4. Data Modeling
not necessary to answer business questions

## 5. Data Evaluation and Visualization

a) Most booked neighborhood in Seattle
